# Calculate noise and dice

In [14]:
config = {}
config['images'] = '/processed/Public/TCIA'
config['gt'] = '/processed/Public/TCIA/labels-vista3d'
config['predictions'] = '/processed/Public/TCIA/predictions/TotalSegmentator_mapped'
config['labels_json'] = 'labels-TCIA-pediatric.json'

In [15]:
# !pip install pyradiomics

In [16]:
import json
import tqdm
import pandas as pd
with open(config['labels_json'],'r') as jsonfile:
    config['labels'] = json.load(jsonfile)

In [17]:
from radiomics import featureextractor

In [18]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import laplace
from radiomics import featureextractor
import os 

def estimate_noise_mad(volume):
    """
    Estimate noise using Median Absolute Deviation (MAD).
    """
    pixels = volume.flatten()
    median_val = np.median(pixels)
    mad = np.median(np.abs(pixels - median_val))
    sigma_est = 1.4826 * mad
    return sigma_est

def estimate_noise_laplacian(volume):
    """
    Estimate noise using the Laplacian operator.
    """
    laplacian_vol = laplace(volume)
    # Mean absolute value of the Laplacian can correlate with noise level
    return np.mean(np.abs(laplacian_vol))

# Calculate boundaries for the central 90%
# Remove 5% on each side for each dimension
def central_bounds(dim_size, fraction=0.05):
    start = int(dim_size * fraction)
    end = int(dim_size * (1 - fraction))
    return start, end

def estimate_noise_pyradiomics(image, mask_path=None):
    """
    Use PyRadiomics to extract a feature as a proxy for noise estimation.
    If no mask is provided, use the entire image as ROI.
    """
    # Initialize radiomics feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()

    # Create a full-volume mask if none provided
    if mask_path:
        mask = sitk.ReadImage(mask_path)
    else:
        # Create an array of ones with the same shape as the image
        array = sitk.GetArrayFromImage(image)
        shape = array.shape
        # For 3D images, shape = (depth, height, width)
        # For 2D images, shape = (height, width)
        # We handle both cases generically.
        if len(shape) == 3:
            z_start, z_end = central_bounds(shape[0])
            y_start, y_end = central_bounds(shape[1])
            x_start, x_end = central_bounds(shape[2])
        elif len(shape) == 2:
            y_start, y_end = central_bounds(shape[0])
            x_start, x_end = central_bounds(shape[1])
        else:
            raise ValueError("Image dimensionality not supported.")

        # Create a mask array of zeros with the same shape as the image
        mask_array = np.zeros_like(array, dtype=np.uint8)

        # Set ones for the central 90% region
        if len(shape) == 3:
            mask_array[z_start:z_end, y_start:y_end, x_start:x_end] = 1
        elif len(shape) == 2:
            mask_array[y_start:y_end, x_start:x_end] = 1
        mask = sitk.GetImageFromArray(mask_array)
        mask.CopyInformation(image)

    # Extract features using PyRadiomics
    result = extractor.execute(image, mask)

    # Use RootMeanSquared as a proxy feature related to intensity variation (noise)
    # (Replace with a more appropriate feature or process as needed)
    noise_feature = result.get('original_firstorder_RootMeanSquared', None)
    return noise_feature

In [19]:
import numpy as np
from scipy.ndimage import uniform_filter
from scipy.ndimage import generic_filter
from scipy.signal import convolve
from scipy.ndimage import histogram

def get_global_noise_index(dcmimg, range_vals=None, rsize=None):
    """
    Calculate the global noise index for medical images.

    Parameters:
    dcmimg (ndarray): The input medical image (2D array).
    range_vals (list or tuple, optional): The range for average intensity filtering. Default is [-300, 100].
    rsize (int, optional): The size of the filter window. Default is 9.

    Returns:
    tuple: (noise, count)
        noise (float): The calculated global noise index.
        count (int): The number of significant bins in the histogram.
    """
    try:
        if range_vals is None:
            range_vals = [-300, 100]
        if rsize is None:
            rsize = 9
        
        # Ensure rsize is an odd number
        rsize = (rsize // 2) * 2 + 1
        
        # Convert dcmimg to float if it's not already
        if not np.issubdtype(dcmimg.dtype, np.floating):
            dcmimg = dcmimg.astype(float)
        
        # Create an averaging filter
        avg = uniform_filter(dcmimg, size=rsize, mode='constant', cval=np.nan)
        
        # Compute local standard deviation
        def local_std(window):
            return np.std(window)
        
        var = generic_filter(dcmimg, local_std, size=rsize)

        # Set boundary pixels to NaN
        pad_size = rsize // 2

        var[:pad_size, :] = np.nan
        var[-pad_size:, :] = np.nan
        var[:, :pad_size] = np.nan
        var[:, -pad_size:] = np.nan
        
        
        # Mask out values outside the range
        var[(avg < range_vals[0]) | (avg > range_vals[1])] = np.nan
        
        # Create histogram
        t = 0.1
        bins = np.arange(0, 200 + t, t)
        hist_vals, bin_edges = np.histogram(var[~np.isnan(var)], bins=bins)
        
        # Smooth histogram
        smooth_filter = np.ones(int(round(2 / t))) / int(round(2 / t))
        hist_vals = convolve(hist_vals, smooth_filter, mode='same')
        
        # Find significant bins
        max_val = np.max(hist_vals)
        significant_indices = np.where(hist_vals > 0.90 * max_val)[0]
        

        
        if len(significant_indices) > 1:
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Compute bin centers
            g = bin_centers[significant_indices] 

            # Calculate noise
            h = hist_vals[significant_indices]
   
            noise = np.sum(g * h) / np.sum(h)

            count = len(significant_indices)
            return noise, count
            
        # Default return if no significant bins found
        return np.nan, 0
    
    except Exception as e:
        print(f"Error: {e}")
        return np.nan, 0

In [20]:
from joblib import Parallel, delayed
import numpy as np

# parallel execution
def get_global_noise_index_3D(image_3d, n_jobs=-1):
    """
    Calculate the global noise index for a 3D image by averaging across slices.

    Parameters:
    image_3d (ndarray): 3D medical image.
    n_jobs (int): Number of parallel jobs to run. Default is -1 (use all available cores).

    Returns:
    float: Average global noise index across all slices.
    """
    # Define a function to calculate noise for a single slice
    def noise_for_slice(slice_2d):
        noise, _ = get_global_noise_index(slice_2d)
        return noise

    # Parallelize noise calculation across slices
    noise_3d = Parallel(n_jobs=n_jobs)(
        delayed(noise_for_slice)(image_3d[:, :, i]) for i in range(image_3d.shape[2])
    )

    # Average noise across slices
    return np.nanmean(noise_3d)


# single core execution
# def get_global_noise_index_3D(image_3d):
#     """
#     Calculate the global noise index for a 3D image by averaging across slices.

#     Parameters:
#     image_3d (ndarray): 3D medical image with shape (X, Y, Z).

#     Returns:
#     float: Average global noise index across all slices.
#     """
#     # Validate input
#     if not isinstance(image_3d, np.ndarray) or image_3d.ndim != 3:
#         raise ValueError("Input image must be a 3D NumPy array.")

#     # Define a function to calculate noise for a single slice
#     def noise_for_slice(slice_2d):
#         noise, _ = get_global_noise_index(slice_2d)
#         return noise

#     # Initialize a list to store noise values for each slice
#     noise_3d = []

#     # Iterate over each slice along the third dimension (Z-axis)
#     num_slices = image_3d.shape[2]
#     for i in range(num_slices):
#         slice_2d = image_3d[:, :, i]
#         try:
#             noise = noise_for_slice(slice_2d)
#             noise_3d.append(noise)
#         except Exception as e:
#             print(f"Error processing slice {i}: {e}")
#             noise_3d.append(np.nan)  # Append NaN or another sentinel value

#     # Convert the list to a NumPy array for numerical operations
#     noise_3d = np.array(noise_3d)

#     # Calculate and return the mean noise, ignoring NaN values
#     average_noise = np.nanmean(noise_3d)
#     return average_noise


In [21]:
import SimpleITK as sitk
import numpy as np

def sitk_to_numpy(image):
    """Convert a SimpleITK image to a numpy array."""
    return sitk.GetArrayFromImage(image)

def dice_coefficient(true_array, pred_array, label):
    """
    Calculate Dice coefficient for a specific label.
    Also return intersection and sum of sizes for micro-average computation.
    """
    true_bin = (true_array == label)
    pred_bin = (pred_array == label)

    intersection = np.logical_and(true_bin, pred_bin).sum()
    sum_sizes = true_bin.sum() + pred_bin.sum()

    # Handle case of no voxels for label in both images
    dice = 1.0 if sum_sizes == 0 else 2.0 * intersection / sum_sizes
    return dice, intersection, sum_sizes

def multi_class_dice(true_image, pred_image, labels):
    """
    Compute label-wise, macro-average, and micro-average Dice for multiple labels.
    
    Parameters:
        true_image (sitk.Image): Ground truth segmentation.
        pred_image (sitk.Image): Predicted segmentation.
        labels (list or iterable): List of label values to compute Dice for.
        
    Returns:
        dict: Dice scores for each label.
        float: Macro-average Dice score.
        float: Micro-average Dice score.
    """
    true_array = sitk_to_numpy(true_image)
    pred_array = sitk_to_numpy(pred_image)

    dice_scores = {}
    macro_sum = 0.0

    # For micro-average calculations
    total_intersection = 0
    total_sum_sizes = 0
    print('Comes here')
    for key,label in labels.items():
        dice, intersection, sum_sizes = dice_coefficient(true_array, pred_array, label)
        dice_scores[key] = dice
        macro_sum += dice

        total_intersection += intersection
        total_sum_sizes += sum_sizes

    macro_average_dice = macro_sum / len(labels) if labels else float('nan')
    # Handle micro-average division by zero
    micro_average_dice = 1.0 if total_sum_sizes == 0 else 2.0 * total_intersection / total_sum_sizes

    return dice_scores, macro_average_dice, micro_average_dice


In [22]:
# Make sure label exists
ct_images = [f for f in os.listdir(config['gt']) if f.endswith('.nii.gz')]

results = []
for ct_name in tqdm.tqdm(ct_images):
    try:
        ct_image_path = os.path.join(config['images'],ct_name)
        ct_image = sitk.ReadImage(ct_image_path)
        ct_volume = sitk.GetArrayFromImage(ct_image)

        # Calculate noise estimates.
#         mad_noise = estimate_noise_mad(ct_volume)
#         laplace_noise = estimate_noise_laplacian(ct_volume)
#         pyradiomics_noise = 0 #estimate_noise_pyradiomics(ct_image)
        ct_volume = np.transpose(ct_volume,(2,0,1))
        global_noise = get_global_noise_index_3D(ct_volume)
        print('global noise calculated')
        label_path = os.path.join(config['gt'],ct_name)
        pred_path = os.path.join(config['predictions'],ct_name)
        label_image = sitk.ReadImage(label_path)
        pred_image = sitk.ReadImage(pred_path)

        dice_scores, macro_avg, micro_avg = multi_class_dice(label_image, pred_image, config['labels'])

        # Prepare record for current CT
        record = {
            'ct_name': ct_name,
#             'mad_noise': mad_noise,
#             'laplace_noise': laplace_noise,
#             'pyradiomics_noise': pyradiomics_noise,
            'global_noise': global_noise,
            'TS_macro_avg_dice': macro_avg,
            'TS_micro_avg_dice': micro_avg
        }

        # Add organ-specific Dice scores to the record
        for organ, dice_val in dice_scores.items():
            record[f'TS_dice_{organ}'] = dice_val

        results.append(record)
    except Exception as e:
        print(e)

  0% 0/358 [00:00<?, ?it/s]

global noise calculated
Comes here


  0% 1/358 [00:44<4:21:48, 44.00s/it]

global noise calculated
Comes here


  1% 2/358 [01:56<6:01:25, 60.91s/it]

...

 99% 356/358 [2:01:39<00:36, 18.36s/it]

global noise calculated
Comes here


100% 357/358 [2:01:52<00:16, 16.92s/it]

global noise calculated
Comes here


100% 358/358 [2:02:03<00:00, 20.46s/it]


In [23]:
# Create a DataFrame from results and save to CSV
df = pd.DataFrame(results)
output_csv_path = "ct_global_noise_dice_results_TS.csv"  # Specify desired output path
df.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")

Results saved to ct_global_noise_dice_results_TS.csv


In [24]:
display(df)

,ct_name,global_noise,TS_macro_avg_dice,TS_micro_avg_dice,TS_dice_right adrenal gland,TS_dice_left adrenal gland,TS_dice_bladder,TS_dice_duodenum,TS_dice_esophagus,TS_dice_gallbladder,TS_dice_heart,TS_dice_right kidney,TS_dice_left kidney,TS_dice_liver,TS_dice_pancreas,TS_dice_prostate,TS_dice_small bowel,TS_dice_spleen,TS_dice_stomach
0,07-14-2009-NA-CT-24459.nii.gz,10.942230,0.716550,0.888072,0.497130,0.288176,0.931745,0.446619,0.666888,0.749705,0.914140,0.957545,0.962716,0.959263,0.830110,0.000000,0.714665,0.914308,0.915242
1,08-04-2004-NA-CT-80302.nii.gz,12.092809,0.818445,0.937142,0.659578,0.722170,0.944989,0.658029,0.773350,0.822669,0.828141,0.958932,0.956687,0.969420,0.896137,0.254133,0.917758,0.964769,0.949912
2,08-25-2004-NA-CT-14591.nii.gz,11.962138,0.752416,0.888367,0.770954,0.660061,0.947592,0.394044,0.787122,0.893538,0.848921,0.963234,0.963585,0.973744,0.243451,0.411613,0.826262,0.973354,0.628766
3,11-22-2005-NA-CT-47368.nii.gz,11.097726,0.811561,0.894510,0.356746,0.326534,0.939866,0.737323,0.744604,0.891905,0.846797,0.934509,0.942557,0.960146,0.821921,1.000000,0.773610,0.947304,0.949598
4,10-11-2008-NA-CT-72580.nii.gz,7.984895,0.628159,0.835474,0.180905,0.000000,0.704520,0.622451,0.632875,0.853194,0.827259,0.961368,0.936364,0.956354,0.244277,0.000000,0.641457,0.924307,0.937060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,12-31-2007-NA-CT-34710.nii.gz,11.108545,0.783708,0.860057,0.466525,0.395582,0.910108,0.623183,0.703254,0.810673,0.856176,0.939700,0.949388,0.958038,0.754096,1.000000,0.632056,0.882466,0.874371
354,09-16-2006-NA-CT-78668.nii.gz,12.883455,0.845003,0.914084,0.539290,0.625513,0.880589,0.710424,0.756651,0.883987,0.839823,0.961356,0.956508,0.970971,0.812568,1.000000,0.837305,0.972953,0.927104
355,07-15-2008-NA-CT-42067.nii.gz,11.448155,0.723154,0.923910,0.443512,0.297832,0.906978,0.590248,0.663115,0.848637,0.820055,0.967760,0.967352,0.975294,0.795634,0.000000,0.783769,0.971893,0.815227
356,09-19-2008-NA-CT-87707.nii.gz,8.655166,0.689995,0.854857,0.238854,0.091743,0.900658,0.623164,0.800500,0.848017,0.857377,0.000000,0.964516,0.955832,0.590235,1.000000,0.622758,0.923368,0.932908


In [25]:
results

[{'ct_name': '07-14-2009-NA-CT-24459.nii.gz',
  'global_noise': 10.94223033824855,
  'TS_macro_avg_dice': 0.7165501225543905,
  'TS_micro_avg_dice': 0.8880718235890548,
  'TS_dice_right adrenal gland': 0.4971302428256071,
  'TS_dice_left adrenal gland': 0.28817574843850957,
  'TS_dice_bladder': 0.9317453560302587,
  'TS_dice_duodenum': 0.4466188538533771,
  'TS_dice_esophagus': 0.6668876367252238,
  'TS_dice_gallbladder': 0.7497050205779903,
  'TS_dice_heart': 0.9141401686566526,
  'TS_dice_right kidney': 0.9575448241189554,
  'TS_dice_left kidney': 0.9627162497830709,
  'TS_dice_liver': 0.9592633289813755,
  'TS_dice_pancreas': 0.8301100423589266,
  'TS_dice_prostate': 0.0,
  'TS_dice_small bowel': 0.7146645423365255,
  'TS_dice_spleen': 0.9143081064186116,
  'TS_dice_stomach': 0.9152417172107733},
 {'ct_name': '08-04-2004-NA-CT-80302.nii.gz',
  'global_noise': 12.092808743744879,
  'TS_macro_avg_dice': 0.8184448540719219,
  'TS_micro_avg_dice': 0.9371419482902605,
  'TS_dice_right ad